In [1]:
# %% [markdown]
# # Rice Disease Classification Prototype (Task 1)
# **Hardware Requirements:** NVIDIA GPU (4GB+ VRAM) or CPU (may be slower)

# %% [code]
# Environment Setup
%pip install tensorflow opencv-python-headless scikit-learn imbalanced-learn

# %% [code]
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
import cv2, os, json

# Verify GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# %% [markdown]
# ## 1. Data Preparation Pipeline

# %% [code]
# Configuration
DATA_PATH = 'Dataset/new_preprocessed_images'
IMG_SIZE = 224
BATCH_SIZE = 32
CLASS_NAMES = sorted(os.listdir(DATA_PATH)) 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 9.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


2025-05-08 22:14:31.993757: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 22:14:31.997358: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 22:14:32.097921: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 22:14:32.134920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746717272.203425  416534 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746717272.21

GPU Available: []


2025-05-08 22:14:34.634616: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
# Custom data loader with real-time augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.  /255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2 
)

# Train generator
train_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    color_mode='rgb'
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    color_mode='rgb'
)

def build_dwscnn(input_shape=(224,224,3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    # Initial feature extraction
    x = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    x = layers.DepthwiseConv2D((3,3), activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    
    # Depthwise block
    x = layers.DepthwiseConv2D((3,3), activation='relu')(x)
    x = layers.Conv2D(64, (1,1), activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    
    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return models.Model(inputs, outputs)

model = build_dwscnn()
model.summary()

# ## 3. Class Balancing Strategy
# Compute class weights
class_counts = {cls: len(os.listdir(os.path.join(DATA_PATH, cls))) for cls in CLASS_NAMES}
total = sum(class_counts.values())
class_weights = {i: total/(len(class_counts)*count) for i, (cls, count) in enumerate(class_counts.items())}

# Custom focal loss implementation
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_sum(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt), axis=-1)
    return focal_loss_fixed

# ## 4. Training Configuration
# Compile model with mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=focal_loss(),
              metrics=['accuracy'])

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# ## 5. Prototype Training
# Train with subset for prototyping
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Reduced for quick iteration
    validation_data=val_generator,
    validation_steps=50,
    epochs=30,
    class_weight=class_weights,
    callbacks=callbacks
)

# ## 6. Local Evaluation
# Plot training history
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Progress')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Progress')
plt.legend()
plt.show()

# Confusion matrix on validation set
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

val_preds = model.predict(val_generator)
val_labels = val_generator.classes
pred_classes = np.argmax(val_preds, axis=1)

print(classification_report(val_labels, pred_classes, target_names=CLASS_NAMES))

plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(val_labels, pred_classes), 
            annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix')
plt.show()
